In [1]:
import pandas as pd
from thefuzz import process
# from fuzzywuzzy import process

events = pd.read_csv('cleaned_data/events.csv')
accidents = pd.read_csv('cleaned_data/accidents.csv')

events['sector'] = events['occupation_description'] + " " + events['sector_description']

sector_description_1 = events['sector'].dropna().unique()
sector_description_2 = (accidents['occupation_description'] + " " + accidents['industry_sector_description'].str.replace('_', ' ')).dropna().unique()

# fuzzy matching
fuzzy_matches = []
for desc2 in sector_description_2:
    match, score = process.extractOne(desc2, sector_description_1)
    fuzzy_matches.append((desc2, match, score))

fuzzy_match_df = pd.DataFrame(fuzzy_matches, columns=['industry_sector_description', 'matched_sector_description', 'match_score'])

fuzzy_match_df

,industry_sector_description,matched_sector_description,match_score
0,facility operation managers transportation,other transportation equipment manufacturing W...,86
1,"motor vehicle mechanics, technicians and mecha...",elementary and secondary schools Education,86
2,material handlers industrial,general warehousing and storage NEC - Industrial,86
3,labourers in metal fabrication manufacturing,all other miscellaneous fabricated metal produ...,86
4,truck drivers industrial,general warehousing and storage NEC - Industrial,86
...,...,...,...
1370,aerospace engineers transportation,other transportation equipment manufacturing W...,86
1371,other services managers municipal,residential building construction Apartment & ...,86
1372,transportation route and crew schedulers elect...,commercial and institutional building construc...,86
1373,waterworks and gas maintenance workers constru...,commercial and institutional building construc...,86


#### Dates

In [2]:
events['date'] = pd.to_datetime(events['date'], format="%Y-%m-%d")
accidents['date'] = pd.to_datetime(accidents['date'], format="%Y-%m-%d")

### Replacing industry_sector_description with matched_sector_description in accidents dataset

In [3]:
accidents['industry_sector_description'] = accidents['occupation_description'] + " " + accidents['industry_sector_description'].str.replace('_', ' ')
accidents = pd.merge(accidents, fuzzy_match_df[['industry_sector_description', 'matched_sector_description']], how='left', on='industry_sector_description')
accidents = accidents.drop(columns=['occupation_description', 'industry_sector_description']).rename(columns={'matched_sector_description':'sector'})
accidents

,date,worker_age,occupation_category_code,occupation_category_description,NOC,worker_experience_in_years,accident_source_category_description1,source_category_description2,accident_category_description,accident_place_description,city,organization_province_code,sector
0,2023-10-14,58.0,72,FACILITY OPERATION AND MAINTENANCE MANAGERS,722.0,0.5,FLOORS,UNKNOWN,FALL ON SAME LEVEL,PLACE NOT SPECIFIED,markham,ON,other transportation equipment manufacturing W...
1,2023-11-04,29.0,732,MOTOR VEHICLE MECHANICS,7322.0,0.5,TRUCK,"VEHICLE & MOBILE EQUIP. PARTS, N.E.C.",CAUGHT IN OR COMPRESSED BY EQUIP./OBJECTS,FARM.,winnipeg,MB,elementary and secondary schools Education
2,2023-12-17,30.0,745,LONGSHORE WORKERS AND MATERIAL HANDLERS,7453.0,0.5,"PLANT & INDUSTRIAL POWERED VEHICLE, N.E.C.","BODILY MOTION OR POSITION OF INJURED, ILL WORKER",STRUCK BY OBJECT,"RESIDENTIAL INSTITUTION. (HOSPITALS, ORPHANAGE...",harriston,ON,general warehousing and storage NEC - Industrial
3,2023-11-13,64.0,961,"LABOURERS IN PROCESSING, MANUFACTURING AND UTI...",9613.0,0.5,PARKING LOTS,WEATHER AND ATMOSPHERIC CONDITIONS,FALL ON SAME LEVEL,PLACE FOR SPORTS AND RECREATION.,markham,ON,all other miscellaneous fabricated metal produ...
4,2023-11-13,74.0,741,MOTOR VEHICLE AND TRANSIT DRIVERS,7412.0,0.5,OTHER STRUCTURES,WEATHER AND ATMOSPHERIC CONDITIONS,FALL ON SAME LEVEL,PUBLIC BUILDINGS.,newcastle,ON,general warehousing and storage NEC - Industrial
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15426,2022-11-25,51.0,961,"LABOURERS IN PROCESSING, MANUFACTURING AND UTI...",9619.0,0.5,"STAIRS, STEPS","BODILY MOTION OR POSITION OF INJURED, ILL WORKER",FALL TO LOWER LEVEL,HOME.,toronto,ON,hardware manufacturing Wood & Metal Fabrication
15427,2023-07-24,57.0,761,TRADES HELPERS AND LABOURERS,7612.0,0.5,TRACTOR,UNKNOWN,"PEDESTRIAN STRUCK BY VEHICLE, MOBILE EQUIPMENT",INDUSTRIAL SITE.,dunchurch,ON,commercial and institutional building construc...
15428,2023-07-05,60.0,735,STATIONARY ENGINEERS & POWER STATION & SYSTEM...,7353.0,0.5,"BODILY MOTION OR POSITION OF INJURED, ILL WORKER","BODILY MOTION OR POSITION OF INJURED, ILL WORKER",REPETITIVE MOTION,INDUSTRIAL SITE.,ajax,ON,commercial and institutional building construc...
15429,2022-05-05,42.0,341,ASSISTING OCCUPATIONS IN SUPPORT OF HEALTH SER...,3414.0,0.5,HEALTH CARE PATIENT OR RESIDENT OF HEALTH CARE...,UNKNOWN,"ASSAULTS, VIOLENT ACTS AND HARASSMENT BY PERSO...",PLACE NOT SPECIFIED,toronto,ON,iron and steel mills and ferro-alloy manufactu...


### Merging with events dataset

In [4]:
# events['sector'] = events['occupation_description'] + " " + events['sector_description']
# events = events.drop(columns=['occupation_description'])

In [5]:
temp = events.copy()
# temp['date'] = temp['date'].dt.strftime("%Y-%m")
temp['year'] = temp['date'].dt.year
temp['month'] = temp['date'].dt.month
accidents['year'] = accidents['date'].dt.year
accidents['month'] = accidents['date'].dt.month
temp = temp.drop(columns=['date'])
temp = temp.drop_duplicates()
merged = pd.merge(accidents, temp, on=['year', 'month', 'city', 'sector'], how='inner')
merged

,date,worker_age,occupation_category_code,occupation_category_description,NOC,worker_experience_in_years,accident_source_category_description1,source_category_description2,accident_category_description,accident_place_description,...,sector,year,month,program,region,NAICS,occupation_description,sector_code,sector_description,worker_count
0,2023-11-23,45.0,745,LONGSHORE WORKERS AND MATERIAL HANDLERS,7453.0,0.0,"METAL MATERIALS--NONSTRUCTURAL, N.E.C.","SKIDS, PALLETS",STRUCK AGAINST OBJECT,INDUSTRIAL SITE.,...,home health care services Nursing Services,2023,11,Health Care Health & Safety Program,Central West,621610.0,home health care services,NSER,Nursing Services,50 or More
1,2023-06-26,30.0,725,"PLUMBERS, PIPEFITTERS AND GAS FITTERS",7254.0,0.5,"PIPES, DUCTS, TUBING","PERSON--INJURED OR ILL WORKER, N.E.C.",STRUCK BY OBJECT,INDUSTRIAL SITE.,...,residential building construction Apartment & ...,2023,6,Construction Health & Safety Program,Eastern,236110.0,residential building construction,RESM,Apartment & Other Multiple Housing,20 to 49
2,2023-11-24,46.0,341,ASSISTING OCCUPATIONS IN SUPPORT OF HEALTH IND...,3415.0,0.5,FLOORS,"BODILY MOTION OR POSITION OF INJURED, ILL WORKER",FALL ON SAME LEVEL,PLACE NOT SPECIFIED,...,elementary and secondary schools Education,2023,11,Industrial Health & Safety Program,Central East,611110.0,elementary and secondary schools,EDUC,Education,50 or More
3,2023-11-24,46.0,341,ASSISTING OCCUPATIONS IN SUPPORT OF HEALTH IND...,3415.0,0.5,FLOORS,"BODILY MOTION OR POSITION OF INJURED, ILL WORKER",FALL ON SAME LEVEL,PLACE NOT SPECIFIED,...,elementary and secondary schools Education,2023,11,Industrial Health & Safety Program,Central East,611110.0,elementary and secondary schools,EDUC,Education,20 to 49
4,2023-11-27,58.0,725,"PLUMBERS, PIPEFITTERS AND GAS FITTERS",7253.0,0.5,FORKLIFT,STRUCTURAL METAL MATERIALS,CAUGHT IN OR COMPRESSED BY EQUIP./OBJECTS,PLACE NOT SPECIFIED,...,elementary and secondary schools Education,2023,11,Industrial Health & Safety Program,Central East,611110.0,elementary and secondary schools,EDUC,Education,50 or More
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1100,2021-10-30,69.0,666,CLEANERS,6664.0,0.5,FLOORS,LIQUIDS,FALL ON SAME LEVEL,STREET AND HIGHWAY.,...,elementary and secondary schools Education,2021,10,Industrial Health & Safety Program,Central East,611110.0,elementary and secondary schools,EDUC,Education,20 to 49
1101,2021-11-02,60.0,961,"LABOURERS IN PROCESSING, MANUFACTURING AND UTI...",9619.0,19.0,"STAIRS, STEPS","BODILY MOTION OR POSITION OF INJURED, ILL WORKER",FALL TO LOWER LEVEL,PUBLIC BUILDINGS.,...,elementary and secondary schools Education,2021,11,Industrial Health & Safety Program,Western,611110.0,elementary and secondary schools,EDUC,Education,NaN
1102,2021-03-03,57.0,315,NURSE SUPERVISORS AND REGISTERED NURSES,3153.0,15.0,GROUND,WEATHER AND ATMOSPHERIC CONDITIONS,FALL ON SAME LEVEL,HOME.,...,nursing care facilities Homes for Nursing Care,2021,3,Health Care Health & Safety Program,Central West,623110.0,nursing care facilities,HFNC,Homes for Nursing Care,50 or More
1103,2021-07-19,46.0,732,MOTOR VEHICLE MECHANICS,7322.0,3.0,STRUCTURAL METAL MATERIALS,FASTENERS,STRUCK BY OBJECT,PUBLIC BUILDINGS.,...,elementary and secondary schools Education,2021,7,Industrial Health & Safety Program,South Western,611110.0,elementary and secondary schools,EDUC,Education,50 or More


#### Worker Count

In [6]:
import re
def text2num(item:str):
    res = re.findall("\d+", item)
    if len(res) == 1:
        return int(res[0])
    else:
        return (int(res[0]) + int(res[1]))//2

In [7]:
merged = merged.dropna(subset='worker_count')
merged['avg_worker_count'] = merged['worker_count'].apply(text2num)
merged

/var/folders/wn/wdlhry7x1m75kh1md7t0x5p00000gn/T/ipykernel_62595/673307198.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged['avg_worker_count'] = merged['worker_count'].apply(text2num)


,date,worker_age,occupation_category_code,occupation_category_description,NOC,worker_experience_in_years,accident_source_category_description1,source_category_description2,accident_category_description,accident_place_description,...,year,month,program,region,NAICS,occupation_description,sector_code,sector_description,worker_count,avg_worker_count
0,2023-11-23,45.0,745,LONGSHORE WORKERS AND MATERIAL HANDLERS,7453.0,0.0,"METAL MATERIALS--NONSTRUCTURAL, N.E.C.","SKIDS, PALLETS",STRUCK AGAINST OBJECT,INDUSTRIAL SITE.,...,2023,11,Health Care Health & Safety Program,Central West,621610.0,home health care services,NSER,Nursing Services,50 or More,50
1,2023-06-26,30.0,725,"PLUMBERS, PIPEFITTERS AND GAS FITTERS",7254.0,0.5,"PIPES, DUCTS, TUBING","PERSON--INJURED OR ILL WORKER, N.E.C.",STRUCK BY OBJECT,INDUSTRIAL SITE.,...,2023,6,Construction Health & Safety Program,Eastern,236110.0,residential building construction,RESM,Apartment & Other Multiple Housing,20 to 49,34
2,2023-11-24,46.0,341,ASSISTING OCCUPATIONS IN SUPPORT OF HEALTH IND...,3415.0,0.5,FLOORS,"BODILY MOTION OR POSITION OF INJURED, ILL WORKER",FALL ON SAME LEVEL,PLACE NOT SPECIFIED,...,2023,11,Industrial Health & Safety Program,Central East,611110.0,elementary and secondary schools,EDUC,Education,50 or More,50
3,2023-11-24,46.0,341,ASSISTING OCCUPATIONS IN SUPPORT OF HEALTH IND...,3415.0,0.5,FLOORS,"BODILY MOTION OR POSITION OF INJURED, ILL WORKER",FALL ON SAME LEVEL,PLACE NOT SPECIFIED,...,2023,11,Industrial Health & Safety Program,Central East,611110.0,elementary and secondary schools,EDUC,Education,20 to 49,34
4,2023-11-27,58.0,725,"PLUMBERS, PIPEFITTERS AND GAS FITTERS",7253.0,0.5,FORKLIFT,STRUCTURAL METAL MATERIALS,CAUGHT IN OR COMPRESSED BY EQUIP./OBJECTS,PLACE NOT SPECIFIED,...,2023,11,Industrial Health & Safety Program,Central East,611110.0,elementary and secondary schools,EDUC,Education,50 or More,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1098,2022-07-17,22.0,667,"OTHER ATTENDANTS IN TRAVEL, ACCOMMODATION AND ...",6672.0,0.5,GROUND,UNKNOWN,STRUCK AGAINST OBJECT,INDUSTRIAL SITE.,...,2022,7,Construction Health & Safety Program,Central West,236220.0,commercial and institutional building construc...,COMM,Commercial Building Construction,50 or More,50
1100,2021-10-30,69.0,666,CLEANERS,6664.0,0.5,FLOORS,LIQUIDS,FALL ON SAME LEVEL,STREET AND HIGHWAY.,...,2021,10,Industrial Health & Safety Program,Central East,611110.0,elementary and secondary schools,EDUC,Education,20 to 49,34
1102,2021-03-03,57.0,315,NURSE SUPERVISORS AND REGISTERED NURSES,3153.0,15.0,GROUND,WEATHER AND ATMOSPHERIC CONDITIONS,FALL ON SAME LEVEL,HOME.,...,2021,3,Health Care Health & Safety Program,Central West,623110.0,nursing care facilities,HFNC,Homes for Nursing Care,50 or More,50
1103,2021-07-19,46.0,732,MOTOR VEHICLE MECHANICS,7322.0,3.0,STRUCTURAL METAL MATERIALS,FASTENERS,STRUCK BY OBJECT,PUBLIC BUILDINGS.,...,2021,7,Industrial Health & Safety Program,South Western,611110.0,elementary and secondary schools,EDUC,Education,50 or More,50


#### Saving

In [8]:
merged.to_csv("cleaned_data/merged.csv", index=0)

In [ ]:
temp = merged.copy()
temp['date'] = temp['date'].dt.year
temp[['date', 'sector', 'worker_count', 'avg_worker_count']].drop_duplicates().head(20)

,date,sector,worker_count,avg_worker_count
0,2023,home health care services Nursing Services,50 or More,50
1,2023,residential building construction Apartment & ...,20 to 49,34
2,2023,elementary and secondary schools Education,50 or More,50
3,2023,elementary and secondary schools Education,20 to 49,34
5,2023,iron and steel mills and ferro-alloy manufactu...,50 or More,50
6,2023,commercial and institutional building construc...,6 to 19,12
11,2022,elementary and secondary schools Education,20 to 49,34
12,2022,elementary and secondary schools Education,1 to 5,3
14,2022,residential building construction Apartment & ...,50 or More,50
15,2022,elementary and secondary schools Education,50 or More,50
